# Research API: Streaming

Stream real-time progress updates during research, including tool calls and content generation.

**What you'll learn:**
- Enable streaming with `stream=True`
- Parse SSE event types: `tool_call`, `tool_response`, `step_details`, `content`
- Display progress indicators during research

## Setup

In [ ]:
%pip install -U tavily-python --quiet

In [ ]:
import os
import getpass
import json

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY:\n")

In [ ]:
from tavily import TavilyClient
from IPython.display import display, Markdown

client = TavilyClient()

## Stream Research Progress

Process SSE events to show research plan, tool calls, and final report.

In [ ]:
query = "What are the latest developments in artificial intelligence?"
model = "mini"  # "mini" | "pro" | "auto"

print(f"Starting Research: {query}\n")

response = client.research(input=query, model=model, stream=True)
full_report = ""
current_step = None
event_type = None

for chunk in response:
    line = chunk.decode("utf-8").strip() if isinstance(chunk, bytes) else str(chunk).strip()
    if not line:
        continue
    
    # Parse SSE format
    if line.startswith("event:"):
        event_type = line.split("event:", 1)[1].strip()
    elif line.startswith("data:"):
        data = json.loads(line.split("data:", 1)[1].strip())
        
        if event_type == "chat.completion.chunk":
            delta = data.get("choices", [{}])[0].get("delta", {})
            
            # Content chunks (final report)
            if "content" in delta:
                full_report += delta["content"]
                print(delta["content"], end="", flush=True)
            
            # Step details (research progress)
            elif "step_details" in delta:
                step = delta["step_details"]
                step_type = step.get("type", "")
                
                if step_type == "research_plan":
                    print(f"\n📋 Research Plan: {step.get('step', '')}")
                elif step_type == "research":
                    if current_step != step.get("step"):
                        current_step = step.get("step")
                        print(f"\n🔍 Researching: {current_step}")
                elif step_type == "think":
                    print(f"\n💭 Thinking: {step.get('step', '')}")
            
            # Tool calls
            elif "tool_calls" in delta:
                tool_data = delta.get("tool_calls", {})
                if isinstance(tool_data, dict):
                    call_type = tool_data.get("type", "")
                    for item in tool_data.get(call_type, []):
                        if isinstance(item, dict) and item.get("arguments"):
                            print(f"\n🔧 Tool: {item['arguments']}")

print("\n\nResearch Complete!")

In [ ]:
display(Markdown(full_report))

## Next Steps

- See [Polling](./polling.ipynb) for async request tracking
- See [Structured Output](./structured_output.ipynb) for custom response schemas